# Image Fraud Detection Algorithm Analysis

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.image as mpimg
import pandas as pd
import os

DIR = os.getcwd()
IMG_WIDTH = 28
IMG_HEIGHT = 28
IMG_PIXELS = IMG_HEIGHT*IMG_WIDTH
SEED = 123
COLOR_MODE = 'grayscale'

## Data Loading for training

Loading a total of 40000 images with 28x28 pixels and grayscale as the color mode. The division is as the following:

- 20000 real images
- 20000 fake images

In [40]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  DIR+'/Datasets/dataset_train',
  labels = 'inferred',
  label_mode = 'binary',
  seed = SEED,
  color_mode = COLOR_MODE,
  image_size=(IMG_HEIGHT,IMG_WIDTH))

data = train_ds.unbatch()
trainData = list(data.as_numpy_iterator())

Found 40000 files belonging to 2 classes.


We use the tensorflow library to get the data. It's in a form of 4-D numpy array:

- First Dimension: Size 40000 which each element represents the information about the image (pixels and label)
- Second Dimension: First element is the infomation of the pixels and the second is the label corrosponding to the image (real - 1, fake - 0)
- Third Dimension: 28 arrays each one containing 28 elements, each element represents a pixel of the image

In [37]:
print('First Dimension:',len(trainData))
print('Second Dimension\nSize of first element:',trainData[0][0].shape,'\nSecond element:',trainData[0][1])
print('Third Dimension:',len(trainData[0][0][0]))

First Dimension: 42000
Second Dimension
Size of first element: (28, 28, 1) 
Second element: 1
Third Dimension: 28


In [27]:
dataDf = np.zeros((len(trainData),IMG_PIXELS+1))
for i in range(len(trainData)):
    dataDf[i] = np.append(trainData[i][0].flatten()/255,trainData[i][1])
images = pd.DataFrame(data=dataDf,index = [i for i in range(0,len(trainData))],columns = [i for i in range(0,IMG_PIXELS+1)])
images = images.rename(columns= {784:'real'})
images.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,real
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.05098,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.003922,0.003922,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
